In [1]:
# Project Setup and Imports
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import json
import requests
import time
import os
from pathlib import Path
from datetime import datetime, timedelta
from typing import Dict, List, Tuple, Optional, Union
import logging
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import warnings
from dotenv import load_dotenv
import glob

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set random seed for reproducibility
np.random.seed(42)

In [2]:
# Constants and Configuration
PROJECT_ROOT = Path('/Users/joshuacroppenstedt/Desktop/Work/JP Morgan/short_interest_prediction')
DATA_DIR = PROJECT_ROOT / 'data'
FEATURE_DIR = DATA_DIR / 'features'
CACHE_DIR = DATA_DIR / 'cache'
PRICES_DIR = DATA_DIR / 'prices'
FINRA_DIR = DATA_DIR / 'finra'

# Trading day windows (approximations)
WINDOWS = [10, 21, 42, 63, 126, 252]  # 2w, 1m, 2m, 3m, 6m, 12m
WINDOW_NAMES = ['2w', '1m', '2m', '3m', '6m', '12m']
WINDOW_MAPPING = dict(zip(WINDOWS, WINDOW_NAMES))

# Generate timestamp for output files
TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M')

print(f"Configuration:")
print(f"  - Project root: {PROJECT_ROOT}")
print(f"  - Timestamp: {TIMESTAMP}")

Configuration:
  - Project root: /Users/joshuacroppenstedt/Desktop/Work/JP Morgan/short_interest_prediction
  - Timestamp: 20251004_1409


In [8]:
# Loading All Data Collected So Far
print("Loading data...")

# Find the most recent price data file
prices_files = list(PRICES_DIR.glob("unified_price_data_*.parquet"))
if not prices_files:
    raise FileNotFoundError("No unified price data files found.")

# Use the most recent file
prices_file = max(prices_files, key=lambda x: x.stat().st_mtime)
print(f"Loading: {prices_file.name}")

# Find the most recent options data file
options_files = list(FEATURE_DIR.glob("options_data_cleaned_*.parquet"))
if not options_files:
    raise FileNotFoundError("No options data files found.")

# Use the most recent file
options_file = max(options_files, key=lambda x: x.stat().st_mtime)
print(f"Loading: {options_file.name}")

# Find the most recent shares outstanding data file
shares_outstanding_files = list(FEATURE_DIR.glob("shares_outstanding_multiindex_corrected_*.parquet"))
if not shares_outstanding_files:
    raise FileNotFoundError("No shares outstanding data files found.")

# Use the most recent file
shares_outstanding_file = max(shares_outstanding_files, key=lambda x: x.stat().st_mtime)
print(f"Loading: {shares_outstanding_file.name}")

# Find the most recent finra data file
finra_files = list(FINRA_DIR.glob("clean_finra_data_*.parquet"))
if not finra_files:
    raise FileNotFoundError("No FINRA data files found.")

# Use the most recent file
finra_file = max(finra_files, key=lambda x: x.stat().st_mtime)
print(f"Loading: {finra_file.name}")

# Load the price data
prices_data = pd.read_parquet(prices_file)
print(f"Price data shape: {prices_data.shape}")
print(f"Date range: {prices_data.index.min()} to {prices_data.index.max()}")

# Load the options data
options_data = pd.read_parquet(options_file)
print(f"Options data shape: {options_data.shape}")
print(f"Date range: {options_data.index.min()} to {options_data.index.max()}")

# Load the shares outstanding data
shares_outstanding_data = pd.read_parquet(shares_outstanding_file)
print(f"Shares Outstanding data shape: {shares_outstanding_data.shape}")
print(f"Date range: {shares_outstanding_data.index.min()} to {shares_outstanding_data.index.max()}")

# Load the finra data
finra_data = pd.read_parquet(finra_file, engine="pyarrow", memory_map=False, thrift_string_size_limit=2**31 - 1, thrift_container_size_limit=2**31 - 1,)
print(f"FINRA data shape: {finra_data.shape}")
print(f"Date range: {finra_data.index.min()} to {finra_data.index.max()}")

# Extract ticker list
if isinstance(options_data.columns, pd.MultiIndex):
    tickers = sorted(options_data.columns.get_level_values(0).unique().tolist())
else:
    raise ValueError("Expected MultiIndex columns in unified price data")

# Store date range for feature alignment
feature_start_date = options_data.index.min()
feature_end_date = options_data.index.max()
print(f"Feature date range: {feature_start_date} to {feature_end_date}")

Loading data...
Loading: unified_price_data_20250929_174321.parquet
Loading: options_data_cleaned_20251002_172838.parquet
Loading: shares_outstanding_multiindex_corrected_20250929_204539.parquet
Loading: clean_finra_data_20251002_183433.parquet
Price data shape: (2146, 24564)
Date range: 2016-12-29 00:00:00 to 2025-07-15 00:00:00
Options data shape: (182, 32585)
Date range: 2017-12-29 to 2025-07-15
Shares Outstanding data shape: (1803, 2273)
Date range: 2008-03-28 00:00:00 to 2025-08-24 00:00:00
FINRA data shape: (182, 506759)
Date range: 2017-12-29 00:00:00 to 2025-07-15 00:00:00
Feature date range: 2017-12-29 to 2025-07-15


In [9]:
print(prices_data.columns)
print(options_data.columns)
print(shares_outstanding_data.columns)
print(finra_data.columns)

MultiIndex([(    'A', 'adjusted_close'),
            (    'A',          'close'),
            (    'A',           'high'),
            (    'A',            'low'),
            (    'A',           'open'),
            (    'A',         'volume'),
            (   'AA', 'adjusted_close'),
            (   'AA',          'close'),
            (   'AA',           'high'),
            (   'AA',            'low'),
            ...
            ( 'ZUMZ',           'high'),
            ( 'ZUMZ',            'low'),
            ( 'ZUMZ',           'open'),
            ( 'ZUMZ',         'volume'),
            ('ZURVY', 'adjusted_close'),
            ('ZURVY',          'close'),
            ('ZURVY',           'high'),
            ('ZURVY',            'low'),
            ('ZURVY',           'open'),
            ('ZURVY',         'volume')],
           names=['symbol', 'metric'], length=24564)
MultiIndex([(   'A',            'options_atm_call_ask'),
            (   'A',            'options_atm_call_bid

In [14]:
# Fix column order and sort
finra_data = (
    finra_data
    .swaplevel(0, 1, axis=1)      # ('accountingYearMonthNumber', 'A') -> ('A', 'accountingYearMonthNumber')
    .sort_index(axis=1)
)

# prices_data: names are ['symbol','metric']
prices_data.columns = prices_data.columns.set_names(['Ticker', 'Features'])

# options_data: names are ['Ticker','FeatureName']
options_data.columns = options_data.columns.set_names(['Ticker', 'Features'])

# shares_outstanding_data: names are ['Ticker','FeatureName']
shares_outstanding_data.columns = shares_outstanding_data.columns.set_names(['Ticker', 'Features'])

# finra_data: names are [None,'symbolCode']
finra_data.columns = finra_data.columns.set_names(['Ticker', 'Features'])


In [34]:
# Getting the tickers in the options_data that do not have any na values
valid_tickers = [ticker for ticker in tickers if options_data[ticker].isna().sum().sum() == 0]
print(f"Number of valid tickers with non-NA options data: {len(valid_tickers)}")

Number of valid tickers with non-NA options data: 1715


In [62]:
# We first simply create a combined DataFrame with all data,
# we have to first remove all tickers that are not present in the
# tickers list from options_data, then we have to concatenate on
# the finra_data index dates, making sure they are all in the correct
# time format.

# Only keep tickers in tickers
finra_data = finra_data.loc[:, finra_data.columns.get_level_values(0).isin(valid_tickers)]
options_data = options_data.loc[:, options_data.columns.get_level_values(0).isin(valid_tickers)]
shares_outstanding_data = shares_outstanding_data.loc[:, shares_outstanding_data.columns.get_level_values(0).isin(valid_tickers)]
prices_data = prices_data.loc[:, prices_data.columns.get_level_values(0).isin(valid_tickers)]

# Making sure all indices are datetime
finra_data.index = pd.to_datetime(finra_data.index)
options_data.index = pd.to_datetime(options_data.index)
shares_outstanding_data.index = pd.to_datetime(shares_outstanding_data.index)
prices_data.index = pd.to_datetime(prices_data.index)

# Creating a combined DataFrame with all data

dfs = [finra_data, options_data, shares_outstanding_data, prices_data]

combined_data = (
    pd.concat(dfs, axis=1, join='outer')                 # append columns
      .pipe(lambda d: d.groupby(level=[0,1], axis=1).first())  # collapse duplicates
      .sort_index(axis=1)                                # nice ordering: by Ticker, then Features
)

# Keep only rows where we have finra data
combined_data = combined_data.loc[combined_data.index.isin(finra_data.index)]


In [66]:
combined_data.shape

(182, 65308)

In [67]:
# Save the data
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
combined_data_file = f"combined_data_{timestamp}.parquet"
combined_data_path = DATA_DIR / 'combined_data' / combined_data_file
combined_data_path.parent.mkdir(parents=True, exist_ok=True)

combined_data.to_parquet(combined_data_path)

In [68]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 182 entries, 2017-12-29 to 2025-07-15
Columns: 65308 entries, ('A', 'accountingYearMonthNumber') to ('ZUMZ', 'volume')
dtypes: datetime64[ns](1715), float64(56595), object(6998)
memory usage: 90.7+ MB


In [69]:
combined_data["A"].columns

Index(['accountingYearMonthNumber', 'adjusted_close',
       'averageDailyVolumeQuantity', 'changePercent', 'changePreviousNumber',
       'close', 'currentShortPositionQuantity', 'daysToCoverQuantity',
       'file_date', 'high', 'issueName', 'issuerServicesGroupExchangeCode',
       'low', 'marketClassCode', 'open', 'options_atm_call_ask',
       'options_atm_call_bid', 'options_atm_call_iv', 'options_atm_call_price',
       'options_atm_call_volume', 'options_atm_put_ask', 'options_atm_put_bid',
       'options_atm_put_iv', 'options_atm_put_price', 'options_atm_put_volume',
       'options_avg_implied_volatility', 'options_estimated_stock_price',
       'options_put_call_oi_ratio', 'options_put_call_volume_ratio',
       'options_synthetic_short_cost', 'options_total_call_volume',
       'options_total_put_volume', 'options_volume_weighted_call_iv',
       'options_volume_weighted_put_iv', 'previousShortPositionQuantity',
       'revisionFlag', 'shares_outstanding', 'volume'],
     

In [74]:
prices_data.columns

MultiIndex([(   'A', 'adjusted_close'),
            (   'A',          'close'),
            (   'A',           'high'),
            (   'A',            'low'),
            (   'A',           'open'),
            (   'A',         'volume'),
            (  'AA', 'adjusted_close'),
            (  'AA',          'close'),
            (  'AA',           'high'),
            (  'AA',            'low'),
            ...
            ( 'ZTS',           'high'),
            ( 'ZTS',            'low'),
            ( 'ZTS',           'open'),
            ( 'ZTS',         'volume'),
            ('ZUMZ', 'adjusted_close'),
            ('ZUMZ',          'close'),
            ('ZUMZ',           'high'),
            ('ZUMZ',            'low'),
            ('ZUMZ',           'open'),
            ('ZUMZ',         'volume')],
           names=['Ticker', 'Features'], length=10290)

In [ ]:
TRADING_DAYS = 252  # typical number of trading days in a year

def _sma(series: pd.Series, window: int, min_periods: int=None) -> pd.Series:
    """Simple Moving Average (SMA)."""
    if min_periods is None: min_periods = window
    return series.rolling(window=window, min_periods=min_periods).mean()

def _ewma(series: pd.Series, window: int) -> pd.Series:
    """Exponential Weighted Moving Average (EWMA) using span=window."""
    return series.ewm(span=window, adjust=False).mean()

def _z_score(series: pd.Series, window: int, min_periods: int=None) -> pd.Series:
    """Rolling z-score of a series."""
    if min_periods is None: min_periods = window
    m = series.rolling(window, min_periods=min_periods).mean()
    s = series.rolling(window, min_periods=min_periods).std(ddof=0)
    return (series - m) / s

def _logret_1d(adj_close: pd.Series) -> pd.Series:
    """1-day log return on adjusted close."""
    return np.log(adj_close / adj_close.shift(1))

def _momentum(adj_close: pd.Series, window: int) -> pd.Series:
    """k-day momentum (simple return)."""
    return adj_close.pct_change(window)

def _vol_cc_from_logrets(logrets: pd.Series, window: int, annualize: bool=True) -> pd.Series:
    """Close-to-close realized vol over a window (std of logrets)."""
    vol = logrets.rolling(window, min_periods=window).std(ddof=0)
    return vol * np.sqrt(TRADING_DAYS) if annualize else vol

def _vol_ewma_from_logrets(logrets: pd.Series, lam: float=0.94, annualize: bool=True) -> pd.Series:
    """
    EWMA volatility with decay lambda (RiskMetrics style).
    lam=0.94 daily is common; returns annualized std.
    """
    var = (1 - lam) * (logrets ** 2)
    # recursive EWMA via ewm with alpha=(1-lam)
    ewvar = var.ewm(alpha=(1 - lam), adjust=False).mean()
    vol = np.sqrt(ewvar)
    return vol * np.sqrt(TRADING_DAYS) if annualize else vol

def _vol_yang_zhang(open_: pd.Series, high: pd.Series, low: pd.Series, close: pd.Series,
                    window: int, annualize: bool=True) -> pd.Series:
    """
    Yang–Zhang volatility (robust to drift; combines overnight, intraday, RS).
    """
    c_prev = close.shift(1)
    r_o = np.log(open_ / c_prev)
    r_c = np.log(close / open_)
    u = np.log(high / open_)
    d = np.log(low / open_)
    m = np.log(close / open_)
    rs = u * (u - m) + d * (d - m)

    n = float(window)
    k = 0.34 / (1.0 + (n + 1.0) / max(n - 1.0, 1.0))

    var_o = r_o.rolling(window, min_periods=window).var(ddof=0)
    var_c = r_c.rolling(window, min_periods=window).var(ddof=0)
    rs_mean = rs.rolling(window, min_periods=window).mean()

    yz_var = var_o + k * var_c + (1.0 - k) * rs_mean
    vol = np.sqrt(np.clip(yz_var, 0, None))
    return vol * np.sqrt(TRADING_DAYS) if annualize else vol

def _downside_semivol(logrets: pd.Series, window: int, annualize: bool=True) -> pd.Series:
    """Std of negative log returns only, over a window."""
    neg = logrets.where(logrets < 0, 0.0)
    # unbiased-ish: treat zeros from positives as 0 contribution
    var = (neg ** 2).rolling(window, min_periods=window).mean()
    vol = np.sqrt(var)
    return vol * np.sqrt(TRADING_DAYS) if annualize else vol

def _vol_of_vol(vol_series: pd.Series, window: int) -> pd.Series:
    """Std of daily vol estimates over a window (regime change proxy)."""
    return vol_series.rolling(window, min_periods=window).std(ddof=0)

def _true_range(high: pd.Series, low: pd.Series, close: pd.Series) -> pd.Series:
    """Wilder's True Range."""
    prev_close = close.shift(1)
    range1 = high - low
    range2 = (high - prev_close).abs()
    range3 = (low - prev_close).abs()
    return pd.concat([range1, range2, range3], axis=1).max(axis=1)

def _atr(high: pd.Series, low: pd.Series, close: pd.Series, window: int) -> pd.Series:
    """Average True Range (ATR)."""
    tr = _true_range(high, low, close)
    return tr.rolling(window, min_periods=window).mean()

def _gap(open_: pd.Series, close: pd.Series) -> pd.Series:
    """Overnight gap as %: (O_t - C_{t-1})/C_{t-1}."""
    return (open_ - close.shift(1)) / close.shift(1)

def _gap_zscore(open_: pd.Series, close: pd.Series, window: int) -> pd.Series:
    """Z-score of the overnight gap."""
    g = _gap(open_, close)
    return _z_score(g, window)

def _dollar_volume(close: pd.Series, volume: pd.Series) -> pd.Series:
    return close.astype(float) * volume.astype(float)

def _adv(volume: pd.Series, window: int) -> pd.Series:
    """Average Daily Volume over window."""
    return volume.rolling(window, min_periods=window).mean()

def _volume_shock_ratio(volume: pd.Series, adv: pd.Series) -> pd.Series:
    """Volume / ADV."""
    return volume / adv.replace(0, np.nan)

def _volume_zscore(volume: pd.Series, window: int) -> pd.Series:
    """Z-score of raw volume."""
    return _z_score(volume.astype(float), window)

def _amihud_illiq(abs_ret: pd.Series, dollar_vol: pd.Series, window: int) -> pd.Series:
    """
    Amihud illiquidity: mean(|ret| / dollar_volume) over window.
    """
    ratio = abs_ret / dollar_vol.replace(0, np.nan)
    return ratio.rolling(window, min_periods=window).mean()

def _rolling_skew(logrets: pd.Series, window: int) -> pd.Series:
    return logrets.rolling(window, min_periods=window).skew()

def _rolling_kurt(logrets: pd.Series, window: int) -> pd.Series:
    return logrets.rolling(window, min_periods=window).kurt()

def _jump_proxy(logrets: pd.Series, vol_cc: pd.Series, c: float=3.0) -> pd.Series:
    """
    Simple jump indicator: (|r| - c * daily_vol)+
    Use c in [2.5, 4] typically. Returns excess magnitude; 0 if none.
    """
    return (logrets.abs() - c * (vol_cc / np.sqrt(TRADING_DAYS))).clip(lower=0)

def _acf1(series: pd.Series, window: int) -> pd.Series:
    """
    Rolling lag-1 autocorrelation.
    """
    x = series
    xm = x.rolling(window, min_periods=window).mean()
    x0 = x - xm
    x1 = x.shift(1) - xm.shift(1)
    num = (x0 * x1).rolling(window, min_periods=window).mean()
    den = (x0 ** 2).rolling(window, min_periods=window).mean()
    return num / den

def _dist_to_rolling_extrema(price: pd.Series, window: int) -> pd.Series:
    """
    Distance to rolling extrema as two-column DataFrame: % to max and % to min.
    """
    roll_max = price.rolling(window, min_periods=window).max()
    roll_min = price.rolling(window, min_periods=window).min()
    dist_high = (price - roll_max) / roll_max
    dist_low  = (price - roll_min) / roll_min
    return pd.DataFrame({'dist_to_high': dist_high, 'dist_to_low': dist_low})

def _price_to_ma(price: pd.Series, ma: pd.Series) -> pd.Series:
    """Scale-free level: price / MA."""
    return price / ma.replace(0, np.nan)

In [79]:
def features_from_prices_df_multi(df: pd.DataFrame, windows: list[int]) -> pd.DataFrame:
    """
    Build a model-ready feature table for a SINGLE TICKER OHLCV frame (indexed by date),
    computing only features with strong intuition for 2-week short-interest prediction.

    Required columns: ['open','high','low','close','adjusted_close','volume'].
    Returns a DataFrame aligned to df.index with columns for all requested windows,
    plus a few single-calculation features.
    """
    # --- unpack once & shared primitives ---
    o, h, l, c = df['open'], df['high'], df['low'], df['close']
    ac, v = df['adjusted_close'], df['volume']

    logr = _logret_1d(ac)
    ret1 = ac.pct_change()
    dvol = _dollar_volume(c, v)

    # Non-windowed items we still want
    out_single = pd.DataFrame({
        'vol_ewma94': _vol_ewma_from_logrets(logr, lam=0.94),
        'dollar_volume': dvol,
    }, index=df.index)

    per_window_frames = []

    for w in windows:
        # level/Trend
        sma = _sma(ac, w)
        ema = _ewma(ac, w)
        price_z = _z_score(ac, w)
        price_to_ma = _price_to_ma(ac, sma)
        mom = _momentum(ac, w)

        # distance to rolling extrema
        dist_ext = _dist_to_rolling_extrema(ac, w).rename(
            columns={'dist_to_high': f'dist_to_high_{w}', 'dist_to_low': f'dist_to_low_{w}'}
        )

        # volatility set
        vol_cc = _vol_cc_from_logrets(logr, w)                  # annualized
        vol_yz = _vol_yang_zhang(o, h, l, c, w)                 # annualized
        semivol = _downside_semivol(logr, w)                    # annualized
        vov = _vol_of_vol(vol_cc, w)                            # regime-change proxy
        atr = _atr(h, l, c, w)

        # volume pressure & liquidity
        adv = _adv(v, w)
        vol_shock = _volume_shock_ratio(v, adv)
        vol_z = _volume_zscore(v, w)
        amihud = _amihud_illiq(ret1.abs(), dvol, w)

        # tails & news proxies
        skew = _rolling_skew(logr, w)
        kurt = _rolling_kurt(logr, w)
        jump = _jump_proxy(logr, vol_cc, c=3.0)
        gap_z = _gap_zscore(o, c, w)

        frame_w = pd.DataFrame({
            f'sma_{w}': sma,
            f'ema_{w}': ema,
            f'price_z_{w}': price_z,
            f'price_to_sma_{w}': price_to_ma,
            f'mom_{w}': mom,
            f'vol_cc_{w}': vol_cc,
            f'vol_yz_{w}': vol_yz,
            f'semivol_{w}': semivol,
            f'vov_{w}': vov,
            f'atr_{w}': atr,
            f'adv_{w}': adv,
            f'vol_shock_{w}': vol_shock,
            f'volume_z_{w}': vol_z,
            f'amihud_{w}': amihud,
            f'skew_{w}': skew,
            f'kurt_{w}': kurt,
            f'jump_{w}': jump,
            f'gap_z_{w}': gap_z,
        }, index=df.index)

        # add extremes (already named)
        frame_w = pd.concat([frame_w, dist_ext], axis=1)

        per_window_frames.append(frame_w)

    # concat all windows + singletons
    out = pd.concat([out_single] + per_window_frames, axis=1)

    return out

In [84]:
combined_price_features_data = pd.DataFrame()

for ticker in valid_tickers:
    print(f"Processing ticker: {ticker}")
    price_df = prices_data[ticker]
    if price_df.empty:
        print(f"  - No valid price data for {ticker}, skipping.")
        continue
    features_df = features_from_prices_df_multi(price_df, WINDOWS)
    # Prefix columns with ticker for multiindex
    features_df.columns = pd.MultiIndex.from_product([[ticker], features_df.columns])
    combined_price_features_data = pd.concat([combined_price_features_data, features_df], axis=1)

Processing ticker: A
Processing ticker: AA
Processing ticker: AAL
Processing ticker: AAOI
Processing ticker: AAON
Processing ticker: AAPL
Processing ticker: AAT
Processing ticker: ABBV
Processing ticker: ABCB
Processing ticker: ABG
Processing ticker: ABM
Processing ticker: ABR
Processing ticker: ABT
Processing ticker: ABUS
Processing ticker: ACAD
Processing ticker: ACCO
Processing ticker: ACGL
Processing ticker: ACHC
Processing ticker: ACIW
Processing ticker: ACLS
Processing ticker: ACM
Processing ticker: ACN
Processing ticker: ACRE
Processing ticker: ACRS
Processing ticker: ACTG
Processing ticker: ADAP
Processing ticker: ADBE
Processing ticker: ADC
Processing ticker: ADI
Processing ticker: ADM
Processing ticker: ADP
Processing ticker: ADSK
Processing ticker: ADTN
Processing ticker: ADUS
Processing ticker: ADVM
Processing ticker: AEE
Processing ticker: AEIS
Processing ticker: AEM
Processing ticker: AEO
Processing ticker: AEP
Processing ticker: AER
Processing ticker: AES
Processing tick

A                                                   \
           vol_ewma94 dollar_volume      sma_10      ema_10 price_z_10   
date                                                                     
2016-12-29        NaN  4.068340e+07         NaN   42.771234        NaN   
2016-12-30   0.006822  5.540552e+07         NaN   42.757603        NaN   
2017-01-03   0.020351  8.087986e+07         NaN   42.904913        NaN   
2017-01-04   0.023313  8.578153e+07         NaN   43.129377        NaN   
2017-01-05   0.025311  6.998513e+07         NaN   43.217611        NaN   
...               ...           ...         ...         ...        ...   
2025-07-09   0.061298  2.191580e+08  119.829356  119.776570   1.201540   
2025-07-10   0.063467  2.818082e+08  120.408729  120.575376   2.233716   
2025-07-11   0.061914  1.704927e+08  120.819266  121.067125   1.346661   
2025-07-14   0.065045  2.436098e+08  120.934752  120.887648  -0.491731   
2025-07-15   0.085943  4.462099e+08  120.452000  119.442621  -2.627001   

                                                                               \
           price_to_sma_10    mom_10 vol_cc_10 vol_yz_10 semivol_10    vov_10   
date                                                                            
2016-12-29             NaN       NaN       NaN       NaN        NaN       NaN   
2016-12-30             NaN       NaN       NaN       NaN        NaN       NaN   
2017-01-03             NaN       NaN       NaN       NaN        NaN       NaN   
2017-01-04             NaN       NaN       NaN       NaN        NaN       NaN   
2017-01-05             NaN       NaN       NaN       NaN        NaN       NaN   
...                    ...       ...       ...       ...        ...       ...   
2025-07-09        1.012273  0.033235  0.172928  0.285590   0.102080  0.021758   
2025-07-10        1.031238  0.048943  0.197206  0.307971   0.102080  0.022118   
2025-07-11        1.020367  0.034448  0.204741  0.306213   0.108279  0.023151   
2025-07-14        0.992932  0.009711  0.248785  0.310954   0.170424  0.031631   
2025-07-15        0.937635 -0.040992  0.387301  0.367693   0.348327  0.069154   

                                                                       \
             atr_10     adv_10 vol_shock_10 volume_z_10     amihud_10   
date                                                                    
2016-12-29      NaN        NaN          NaN         NaN           NaN   
2016-12-30      NaN        NaN          NaN         NaN           NaN   
2017-01-03      NaN        NaN          NaN         NaN           NaN   
2017-01-04      NaN        NaN          NaN         NaN           NaN   
2017-01-05      NaN        NaN          NaN         NaN           NaN   
...             ...        ...          ...         ...           ...   
2025-07-09  2.96837  1604710.3     1.125900    0.732373  4.760167e-11   
2025-07-10  3.23687  1662631.3     1.365026    1.780311  5.184285e-11   
2025-07-11  3.17037  1652129.3     0.837084   -0.774186  5.225152e-11   
2025-07-14  3.36837  1681531.9     1.206477    0.950215  6.189407e-11   
2025-07-15  3.96187  1916368.1     2.061638    2.642597  7.007246e-11   

                                                                  \
             skew_10   kurt_10 jump_10  gap_z_10 dist_to_high_10   
date                                                               
2016-12-29       NaN       NaN     NaN       NaN             NaN   
2016-12-30       NaN       NaN     NaN       NaN             NaN   
2017-01-03       NaN       NaN     NaN       NaN             NaN   
2017-01-04       NaN       NaN     NaN       NaN             NaN   
2017-01-05       NaN       NaN     NaN       NaN             NaN   
...              ...       ...     ...       ...             ...   
2025-07-09 -0.257724  0.410577     0.0  0.595511       -0.000659   
2025-07-10 -0.112416 -0.395782     0.0  0.344131        0.000000   
2025-07-11  0.189611 -0.815603     0.0 -1.902705       -0.007168   
2025-07-1

In [85]:
print(combined_price_features_data.shape)

(2146, 209230)


In [86]:
# Save the data
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
combined_price_features_data_file = f"combined_price_features_data_{timestamp}.parquet"
combined_price_features_data_path = FEATURE_DIR / combined_price_features_data_file
combined_price_features_data_path.parent.mkdir(parents=True, exist_ok=True)

combined_price_features_data.to_parquet(combined_price_features_data_path)

In [89]:
# Creating a combined DataFrame with all data

dfs = [finra_data, options_data, shares_outstanding_data, prices_data, combined_price_features_data]

combined_data_with_features = (
    pd.concat(dfs, axis=1, join='outer')                 # append columns
      .pipe(lambda d: d.groupby(level=[0,1], axis=1).first())  # collapse duplicates
      .sort_index(axis=1)                                # nice ordering: by Ticker, then Features
)

# Keep only rows where we have finra data
combined_data_with_features = combined_data_with_features.loc[combined_data_with_features.index.isin(finra_data.index)]

In [90]:
# Save the data
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
combined_data_with_features_file = f"combined_data_with_features_{timestamp}.parquet"
combined_data_with_features_path = DATA_DIR / 'combined_data' / combined_data_with_features_file
combined_data_with_features_path.parent.mkdir(parents=True, exist_ok=True)

combined_data_with_features.to_parquet(combined_data_with_features_path)

In [91]:
columns_to_keep = []

combined_data_with_features.shape

(182, 274538)

In [94]:
combined_data["A"].columns

Index(['accountingYearMonthNumber', 'adjusted_close',
       'averageDailyVolumeQuantity', 'changePercent', 'changePreviousNumber',
       'close', 'currentShortPositionQuantity', 'daysToCoverQuantity',
       'file_date', 'high', 'issueName', 'issuerServicesGroupExchangeCode',
       'low', 'marketClassCode', 'open', 'options_atm_call_ask',
       'options_atm_call_bid', 'options_atm_call_iv', 'options_atm_call_price',
       'options_atm_call_volume', 'options_atm_put_ask', 'options_atm_put_bid',
       'options_atm_put_iv', 'options_atm_put_price', 'options_atm_put_volume',
       'options_avg_implied_volatility', 'options_estimated_stock_price',
       'options_put_call_oi_ratio', 'options_put_call_volume_ratio',
       'options_synthetic_short_cost', 'options_total_call_volume',
       'options_total_put_volume', 'options_volume_weighted_call_iv',
       'options_volume_weighted_put_iv', 'previousShortPositionQuantity',
       'revisionFlag', 'shares_outstanding', 'volume'],
     

In [95]:
prices_data["A"]

Features,adjusted_close,close,high,low,open,volume
date,,,,,,
2016-12-29,42.771234,45.64,46.000,45.5700,45.68,891398.0
2016-12-30,42.696263,45.56,45.820,45.3750,45.76,1216100.0
2017-01-03,43.567806,46.49,46.750,45.7400,45.93,1739726.0
2017-01-04,44.139464,47.10,47.380,46.8162,46.93,1821264.0
2017-01-05,43.614663,46.54,47.070,46.3550,47.05,1503763.0
...,...,...,...,...,...,...
2025-07-09,121.300000,121.30,122.350,118.9000,121.40,1806744.0
2025-07-10,124.170000,124.17,126.435,120.9400,121.63,2269535.0
2025-07-11,123.280000,123.28,123.660,122.3350,122.93,1382971.0


In [ ]:
#finra_data, options_data, shares_outstanding_data, prices_data

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

finra_data_file = f"finra_data_{timestamp}.parquet"
finra_data_path = DATA_DIR / 'final_data' / finra_data_file
finra_data_path.parent.mkdir(parents=True, exist_ok=True)
finra_data.to_parquet(finra_data_path)

options_data_file = f"options_data_{timestamp}.parquet"
options_data_path = DATA_DIR / 'final_data' / options_data_file
options_data_path.parent.mkdir(parents=True, exist_ok=True)
options_data.to_parquet(options_data_path)

shares_outstanding_data_file = f"shares_outstanding_data_{timestamp}.parquet"
shares_outstanding_data_path = DATA_DIR / 'final_data' / shares_outstanding_data_file
shares_outstanding_data_path.parent.mkdir(parents=True, exist_ok=True)
shares_outstanding_data.to_parquet(shares_outstanding_data_path)

prices_data_file = f"prices_data_{timestamp}.parquet"
prices_data_path = DATA_DIR / 'final_data' / prices_data_file
prices_data_path.parent.mkdir(parents=True, exist_ok=True)
prices_data.to_parquet(prices_data_path)



In [98]:
shares_outstanding_data[shares_outstanding_data.index >= "2017-12-29"]

Ticker                      A                 AA                AAL  \
Features   shares_outstanding shares_outstanding shares_outstanding   
date                                                                  
2017-12-29        329000000.0        187000000.0        486625000.0   
2017-12-30        329000000.0        187000000.0        486625000.0   
2017-12-31        329000000.0        187000000.0        479382000.0   
2018-01-01        329000000.0        187000000.0        479382000.0   
2018-01-02        329000000.0        187000000.0        479382000.0   
...                       ...                ...                ...   
2025-08-01        285000000.0        260000000.0        658880000.0   
2025-08-02        285000000.0        260000000.0        658880000.0   
2025-08-03        285000000.0        260000000.0        658880000.0   
2025-08-16        285000000.0        260000000.0        658880000.0   
2025-08-24        285000000.0        260000000.0        658880000.0   

Ticker                   AAOI               AAON               AAPL  \
Features   shares_outstanding shares_outstanding shares_outstanding   
date                                                                  
2017-12-29         20422609.0         53014269.0       5.183586e+09   
2017-12-30         20422609.0         53014269.0       5.157787e+09   
2017-12-31         20155488.0         53004712.0       5.157787e+09   
2018-01-01         20155488.0         53004712.0       5.157787e+09   
2018-01-02         20155488.0         53004712.0       5.157787e+09   
...                       ...                ...                ...   
2025-08-01         42311811.0         82956213.0       1.494818e+10   
2025-08-02         42311811.0         82956213.0       1.494818e+10   
2025-08-03         42311811.0         82956213.0       1.494818e+10   
2025-08-16         42311811.0         82956213.0       1.494818e+10   
2025-08-24         42311811.0         82956213.0       1.494818e+10   

Ticker                    AAT               ABBV               ABCB  \
Features   shares_outstanding shares_outstanding shares_outstanding   
date                                                                  
2017-12-29         64093066.0       1.603000e+09         37553000.0   
2017-12-30         64093066.0       1.603000e+09         37553000.0   
2017-12-31         64103909.0       1.606000e+09         37525000.0   
2018-01-01         64103909.0       1.606000e+09         37525000.0   
2018-01-02         64103909.0       1.606000e+09         37525000.0   
...                       ...                ...                ...   
2025-08-01         76718837.0       1.772000e+09         69030331.0   
2025-08-02         76718837.0       1.772000e+09         69030331.0   
2025-08-03         76718837.0       1.772000e+09         69030331.0   
2025-08-16         76718837.0       1.772000e+09         69030331.0   
2025-08-24         76718837.0       1.772000e+09         69030331.0   

Ticker                    ABG                ABM                ABR  \
Features   shares_outstanding shares_outstanding shares_outstanding   
date                                                                  
2017-12-29         20800000.0         63400000.0         83918117.0   
2017-12-30         20800000.0         63400000.0         83918117.0   
2017-12-31         21000000.0         63400000.0         84416251.0   
2018-01-01         21000000.0         63400000.0         84416251.0   
2018-01-02         21000000.0         63400000.0         84416251.0   
...                       ...                ...                ...   
2025-08-01         19700000.0         62900000.0        206862320.0   
2025-08-02         19700000.0         62900000.0        206862320.0   
2025-08-03         19700000.0         62900000.0        206862320.0   
2025-08-16         19700000.0         62900000.0        206862320.0   
2025-08-24         19700000.0         62900000.0        206862320.0   

Ticker              

In [100]:
combined_data["A"].columns

Index(['accountingYearMonthNumber', 'adjusted_close',
       'averageDailyVolumeQuantity', 'changePercent', 'changePreviousNumber',
       'close', 'currentShortPositionQuantity', 'daysToCoverQuantity',
       'file_date', 'high', 'issueName', 'issuerServicesGroupExchangeCode',
       'low', 'marketClassCode', 'open', 'options_atm_call_ask',
       'options_atm_call_bid', 'options_atm_call_iv', 'options_atm_call_price',
       'options_atm_call_volume', 'options_atm_put_ask', 'options_atm_put_bid',
       'options_atm_put_iv', 'options_atm_put_price', 'options_atm_put_volume',
       'options_avg_implied_volatility', 'options_estimated_stock_price',
       'options_put_call_oi_ratio', 'options_put_call_volume_ratio',
       'options_synthetic_short_cost', 'options_total_call_volume',
       'options_total_put_volume', 'options_volume_weighted_call_iv',
       'options_volume_weighted_put_iv', 'previousShortPositionQuantity',
       'revisionFlag', 'shares_outstanding', 'volume'],
     

In [101]:
columns_to_keep = ['adjusted_close',
                   'averageDailyVolumeQuantity',
                   'changePercent', 
                   'changePreviousNumber',
                   'close', 
                   'currentShortPositionQuantity', 
                   'daysToCoverQuantity',
                   'high',
                   'low',
                   'open',
                   'options_put_call_volume_ratio',
                   'options_synthetic_short_cost',
                   'options_avg_implied_volatility',
                   'shares_outstanding',
                   'volume']

combined_data_with_columns = combined_data.loc[:, combined_data.columns.get_level_values(1).isin(columns_to_keep)]

In [ ]:
# Dictionary with desctiptions of each column
column_descriptions = {
    'adjusted_close': 'Adjusted closing price of the stock, accounting for dividends and stock splits: from AlphaVantage.',
    'averageDailyVolumeQuantity': 'The average number of shares traded daily over a specified period: from FINRA.',
    'changePercent': 'The percentage change in the short interest over two weeks: from FINRA.',
    'changePreviousNumber': 'The absolute change in the short interest over two weeks: from FINRA.',
    'close': 'The closing price of the stock on a given trading day: from AlphaVantage.',
    'currentShortPositionQuantity': 'The current number of shares that are sold short and not yet covered or closed out: from FINRA.',
    'daysToCoverQuantity': 'The number of days it would take to cover all short positions based on average daily trading volume: from FINRA.',
    'high': 'The highest price of the stock during a given trading day: from AlphaVantage.',
    'low': 'The lowest price of the stock during a given trading day: from AlphaVantage.',
    'open': 'The opening price of the stock on a given trading day: from AlphaVantage.',
    'options_put_call_volume_ratio': 'The ratio of the volume of put options to call options traded: from AlphaVantage.',
    'options_synthetic_short_cost': 'The cost associated with creating a synthetic short position using options: from AlphaVantage.',
    'options_avg_implied_volatility': 'The average implied volatility of options, reflecting market expectations of future stock price volatility: from AlphaVantage.',
    'shares_outstanding': 'The total number of shares of a company that are currently owned by all its shareholders, including restricted shares owned by company insiders and institutional investors: from AlphaVantage.',
    'volume': 'The total number of shares traded during a given trading day: from AlphaVantage.'
}

MultiIndex([(   'A',                 'adjusted_close'),
            (   'A',     'averageDailyVolumeQuantity'),
            (   'A',                  'changePercent'),
            (   'A',           'changePreviousNumber'),
            (   'A',                          'close'),
            (   'A',   'currentShortPositionQuantity'),
            (   'A',            'daysToCoverQuantity'),
            (   'A',                           'high'),
            (   'A',                            'low'),
            (   'A',                           'open'),
            ...
            ('ZUMZ',   'currentShortPositionQuantity'),
            ('ZUMZ',            'daysToCoverQuantity'),
            ('ZUMZ',                           'high'),
            ('ZUMZ',                            'low'),
            ('ZUMZ',                           'open'),
            ('ZUMZ', 'options_avg_implied_volatility'),
            ('ZUMZ',  'options_put_call_volume_ratio'),
            ('ZUMZ',   'options_

In [105]:
combined_data_with_columns.isna().sum()#.sum()

Ticker  Features                      
A       adjusted_close                     0
        averageDailyVolumeQuantity         0
        changePercent                      0
        changePreviousNumber               0
        close                              0
                                          ..
ZUMZ    options_avg_implied_volatility     0
        options_put_call_volume_ratio      0
        options_synthetic_short_cost       0
        shares_outstanding                90
        volume                             0
Length: 25725, dtype: int64

In [110]:
combined_data_with_columns["ZUMZ"]["shares_outstanding"]

2017-12-29    24804000.0
2018-01-12           NaN
2018-01-31    24804000.0
2018-02-15           NaN
2018-02-28    24977000.0
                 ...    
2025-05-15           NaN
2025-05-30           NaN
2025-06-13           NaN
2025-06-30    18116000.0
2025-07-15           NaN
Name: shares_outstanding, Length: 182, dtype: float64

In [115]:
shares_outstanding_data["ZUMZ"].iloc[shares_outstanding_data["ZUMZ"].index == "2018-01-07"]#.isna().sum()

Features,shares_outstanding
date,


In [134]:
# For now we will forward fill the shares outstanding data for each ticker
for ticker in combined_data_with_columns.columns.get_level_values(0).unique():
    combined_data_with_columns[ticker]["shares_outstanding"].ffill(inplace=True)

In [135]:
combined_data_with_columns["ZUMZ"]["shares_outstanding"]

2017-12-29    24804000.0
2018-01-12    24804000.0
2018-01-31    24804000.0
2018-02-15    24804000.0
2018-02-28    24977000.0
                 ...    
2025-05-15    18881000.0
2025-05-30    18881000.0
2025-06-13    18881000.0
2025-06-30    18116000.0
2025-07-15    18116000.0
Name: shares_outstanding, Length: 182, dtype: float64

In [136]:
combined_data_with_columns.isna().sum().sum()

np.int64(10801)

In [137]:
combined_data_with_columns.isna().sum()[combined_data_with_columns.isna().sum() > 0]

Ticker  Features          
ACAD    shares_outstanding     78
ADAP    shares_outstanding     32
ADVM    shares_outstanding     54
AER     shares_outstanding    132
AGEN    shares_outstanding     54
                             ... 
WB      shares_outstanding     66
WK      shares_outstanding     54
WPM     shares_outstanding    132
WTI     shares_outstanding     78
XRX     shares_outstanding     20
Length: 148, dtype: int64

In [133]:
valid_tickers[valid_tickers == "ACAD"]

'A'

In [138]:
combined_data_with_columns["ACAD"]

Features,adjusted_close,averageDailyVolumeQuantity,changePercent,changePreviousNumber,close,currentShortPositionQuantity,daysToCoverQuantity,high,low,open,options_avg_implied_volatility,options_put_call_volume_ratio,options_synthetic_short_cost,shares_outstanding,volume
2017-12-29,30.110,1411267.0,1.79,174405.0,30.110,9898590.0,7.01,31.20,30.08000,31.18,0.734503,0.210076,0.263200,NaN,844531.0
2018-01-12,28.370,2066963.0,19.63,1943558.0,28.370,11842148.0,5.73,28.44,27.60000,27.80,0.910197,0.528859,1.491389,NaN,1672751.0
2018-01-31,29.910,2159765.0,4.44,526310.0,29.910,12368458.0,5.73,30.96,29.50000,30.68,0.918332,0.303691,0.953590,NaN,1142554.0
2018-02-15,31.060,1490517.0,-6.26,-774072.0,31.060,11594386.0,7.78,31.20,30.50000,31.00,0.927527,0.150345,-1.044250,NaN,828417.0
2018-02-28,24.915,2742978.0,-10.69,-1239260.0,24.915,10355126.0,3.78,27.77,24.50000,27.77,0.789280,0.849624,5.898372,NaN,11581280.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-15,17.600,2198051.0,-0.12,-17677.0,17.600,15243471.0,6.93,17.64,16.96885,17.20,0.851913,0.109955,-1.118000,167668000.0,2019217.0
2025-05-30,21.570,3719881.0,9.27,1413316.0,21.570,16656787.0,4.48,22.40,21.54000,22.30,0.683383,99.000000,-0.188333,167668000.0,1446509.0
2025-06-13,21.710,1480513.0,-7.33,-1220903.0,21.710,15435884.0,10.43,21.96,21.38040,21.46,0.963026,0.212121,-0.058333,167668000.0,1154100.0
2025-06-30,21.570,2305581.0,-16.91,-2610515.0,21.570,12825369.0,5.56,22.26,21.39000,22.15,0.664775,2.000000,1.272500,167668000.0,1626801.0


In [141]:
tickers_without_coverage = combined_data_with_columns.isna().sum()[combined_data_with_columns.isna().sum() > 0].index.get_level_values(0).unique().tolist()

In [142]:
tickers_without_coverage

['ACAD',
 'ADAP',
 'ADVM',
 'AER',
 'AGEN',
 'ALLT',
 'ALNY',
 'APLE',
 'APO',
 'AQMS',
 'ARCC',
 'ARCT',
 'ASC',
 'ATRC',
 'AUPH',
 'AXSM',
 'AZN',
 'BATRA',
 'BATRK',
 'BB',
 'BCE',
 'BG',
 'BLK',
 'BMO',
 'BNS',
 'BOX',
 'BP',
 'CAPL',
 'CI',
 'CLS',
 'CLSD',
 'CM',
 'CPRX',
 'CRH',
 'CSTM',
 'CSWC',
 'CTMX',
 'CVE',
 'DB',
 'DHT',
 'DIS',
 'DMLP',
 'DVAX',
 'EDIT',
 'ESPR',
 'FDUS',
 'FNV',
 'FOLD',
 'FTS',
 'FUN',
 'GAIN',
 'GALT',
 'GBDC',
 'GEL',
 'GGAL',
 'GLAD',
 'GOGO',
 'HRTX',
 'HRZN',
 'HTGC',
 'HTHT',
 'IAC',
 'INVE',
 'IOVA',
 'IRTC',
 'JD',
 'KPTI',
 'KRO',
 'KURA',
 'LILA',
 'LILAK',
 'MAIN',
 'MANU',
 'MCRB',
 'MNOV',
 'MRCC',
 'MRVL',
 'NG',
 'NL',
 'NMFC',
 'NMR',
 'NTLA',
 'NTNX',
 'NVAX',
 'NVCR',
 'NVS',
 'NWE',
 'NXPI',
 'OCSL',
 'OCUL',
 'OMER',
 'OOMA',
 'ORMP',
 'PANW',
 'PBYI',
 'PFLT',
 'PLSE',
 'PNNT',
 'PSEC',
 'PSTG',
 'QDEL',
 'QTWO',
 'QURE',
 'REFR',
 'RIOT',
 'RNG',
 'RY',
 'SA',
 'SB',
 'SBGI',
 'SCM',
 'SENS',
 'SGMO',
 'SGU',
 'SHOP',
 'SIGI',
 'S

In [143]:
# Remove tickers without coverage
combined_data_with_columns = combined_data_with_columns.loc[:, ~combined_data_with_columns.columns.get_level_values(0).isin(tickers_without_coverage)]

combined_data_with_columns.shape

(182, 23505)

In [145]:
combined_data_with_columns.isna().sum().sum()

np.int64(0)

In [146]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

combined_data_with_columns_file = f"combined_data_with_columns_{timestamp}.parquet"
combined_data_with_columns_path = DATA_DIR / 'final_data' / combined_data_with_columns_file
combined_data_with_columns_path.parent.mkdir(parents=True, exist_ok=True)
combined_data_with_columns.to_parquet(combined_data_with_columns_path)

In [147]:
combined_data_with_columns.columns.get_level_values(0).unique()

Index(['A', 'AA', 'AAL', 'AAOI', 'AAON', 'AAPL', 'AAT', 'ABBV', 'ABCB', 'ABG',
       ...
       'YORW', 'YUM', 'YUMC', 'ZBH', 'ZBRA', 'ZEUS', 'ZG', 'ZION', 'ZTS',
       'ZUMZ'],
      dtype='object', name='Ticker', length=1567)

In [148]:
columns_to_keep = ['adjusted_close',
                   'averageDailyVolumeQuantity',
                   'close', 
                   'currentShortPositionQuantity', 
                   'daysToCoverQuantity',
                   'high',
                   'low',
                   'open',
                   'options_put_call_volume_ratio',
                   'options_synthetic_short_cost',
                   'options_avg_implied_volatility',
                   'shares_outstanding',
                   'volume']

combined_data_pruned = combined_data_with_columns.loc[:, combined_data_with_columns.columns.get_level_values(1).isin(columns_to_keep)]

In [151]:
combined_data_pruned["A"]

Features,adjusted_close,averageDailyVolumeQuantity,close,currentShortPositionQuantity,daysToCoverQuantity,high,low,open,options_avg_implied_volatility,options_put_call_volume_ratio,options_synthetic_short_cost,shares_outstanding,volume
2017-12-29,63.327340,1234014.0,66.97,4197300.0,3.40,67.58,66.930,67.50,0.513802,1.482143,-4.746667,329000000.0,1061239.0
2018-01-12,67.828432,1867541.0,71.73,5137321.0,2.75,71.86,70.500,70.73,0.543195,0.208824,-9.515000,329000000.0,1448144.0
2018-01-31,69.435965,1814702.0,73.43,5357450.0,2.95,74.39,73.240,73.77,0.407551,1.216667,-6.228333,323000000.0,1949108.0
2018-02-15,68.102658,3291327.0,72.02,5464884.0,1.66,74.99,71.220,74.94,0.787268,0.249649,-4.608333,323000000.0,4884299.0
2018-02-28,64.859224,3337573.0,68.59,4135308.0,1.24,70.00,68.570,69.42,0.263327,0.412527,3.591667,323000000.0,2657196.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-05-15,112.169050,1985940.0,112.40,4379116.0,2.21,112.40,108.930,111.28,0.552494,13.630719,10.892333,285000000.0,1873978.0
2025-05-30,111.690036,2650294.0,111.92,4136120.0,1.56,112.91,110.788,112.03,0.372479,0.370121,10.898667,285000000.0,4753647.0
2025-06-13,116.839434,2078685.0,117.08,4110706.0,1.98,118.90,116.450,116.68,0.365271,0.304545,5.633333,285000000.0,1652070.0
2025-06-30,117.767523,1749336.0,118.01,3354818.0,1.92,119.66,117.445,119.19,0.364052,0.183457,4.132000,285000000.0,1602496.0


In [152]:
sp600_tickers = pd.read_csv('sp600_matched_with_peers.csv')#['Ticker'].tolist()

In [154]:
sp600_tickers = sp600_tickers['Symbol'].tolist()

In [155]:
sp600_tickers[:10]

['AAP', 'AAT', 'ABCB', 'ABG', 'ABM', 'ABR', 'ACAD', 'ACHC', 'ACIW', 'ACLS']

In [156]:
# Checking the coverage of the sp600_tickers in our data
tickers_in_data = combined_data_pruned.columns.get_level_values(0).unique().tolist()
sp600_tickers_in_data = [ticker for ticker in sp600_tickers if ticker in tickers_in_data]
print(f"Number of SP600 tickers in data: {len(sp600_tickers_in_data)} out of {len(sp600_tickers)}")

Number of SP600 tickers in data: 391 out of 464


In [157]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

combined_data_pruned_file = f"combined_data_pruned_{timestamp}.parquet"
combined_data_pruned_path = DATA_DIR / 'final_data' / combined_data_pruned_file
combined_data_pruned_path.parent.mkdir(parents=True, exist_ok=True)
combined_data_pruned.to_parquet(combined_data_pruned_path)

In [158]:
combined_data_pruned["A"].columns

Index(['adjusted_close', 'averageDailyVolumeQuantity', 'close',
       'currentShortPositionQuantity', 'daysToCoverQuantity', 'high', 'low',
       'open', 'options_avg_implied_volatility',
       'options_put_call_volume_ratio', 'options_synthetic_short_cost',
       'shares_outstanding', 'volume'],
      dtype='object', name='Features')

In [ ]:
# Dictionary with desctiptions of each column
column_descriptions = {
    'adjusted_close': 'Adjusted closing price of the stock, accounting for dividends and stock splits: from AlphaVantage.',
    'averageDailyVolumeQuantity': 'The number of shares traded over the previous two weeks divided by the number of trading days in those two weeks: from FINRA.',
    'close': 'The closing price of the stock on a given trading day: from AlphaVantage.',
    'currentShortPositionQuantity': 'The current number of shares that are sold short and not yet covered or closed out: from FINRA.',
    'daysToCoverQuantity': 'The number of days it would take to cover all short positions based on average daily trading volume: from FINRA.',
    'high': 'The highest price of the stock during a given trading day: from AlphaVantage.',
    'low': 'The lowest price of the stock during a given trading day: from AlphaVantage.',
    'open': 'The opening price of the stock on a given trading day: from AlphaVantage.',
    'options_put_call_volume_ratio': 'The ratio of the volume of put options to call options traded: from AlphaVantage.',
    'options_synthetic_short_cost': 'The cost associated with creating a synthetic short position using options: from AlphaVantage.',
    'options_avg_implied_volatility': 'The average implied volatility of options, reflecting market expectations of future stock price volatility: from AlphaVantage.',
    'shares_outstanding': 'The total number of shares of a company that are currently owned by all its shareholders, including restricted shares owned by company insiders and institutional investors: from AlphaVantage.',
    'volume': 'The total number of shares traded during a given trading day: from AlphaVantage.'
}